In [1]:
from re import L
from sklearn import metrics
import rank_metric as metrics
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import xml.etree.ElementTree as ET
import matplotlib as plt
import pandas as pd
import numpy as np
import pprint as pp
import tarfile
from sklearn.feature_extraction.text import CountVectorizer
import math
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

## Data Loading


In [2]:
#parsing real patient cases
import pickle
import os
trials = "data/qrels-clinical_trials.txt"
trials = pd.read_csv(trials, sep='\t', names=["query_id", "dummy", "docid", "rel"])
trials = trials.drop("dummy", axis = 1)
trials_to_use = set(trials['docid'])
if not os.path.exists("data/doc_brief_title.bin") or not os.path.exists("data/doc_ids.bin") or not os.path.exists("data/docs_detailed_description.bin") or not os.path.exists("data/docs_brief_summary.bin") or not os.path.exists("data/docs_criteria.bin"):
    #parsing clinical trials
    
    tar = tarfile.open("data/clinicaltrials.gov-16_dec_2015.tgz", "r:gz")
    clinical_trials = {}
    docs_brief_title = {}
    docs_detailed_description={}
    docs_brief_summary={}
    docs_criteria={}
    ids_brief_title = []
    ids_detailed_description=[]
    ids_brief_summary=[]
    ids_criteria=[]

    i = 0
    for element in tar:
        if element.size > 500:
            txt = tar.extractfile(element).read().decode("utf-8", "strict")
            root = ET.fromstring(txt)
            for doc_id in root.iter('nct_id'):
                temp_id = doc_id.text
            if temp_id not in trials_to_use:
                continue
            i+=1
            for brief_title in root.iter('brief_title'):
                id_title=doc_id.text
                docs_brief_title[temp_id]=brief_title.text
                ids_brief_title.append(temp_id)

            
            
            for brief_summary in root.iter('brief_summary'):
                for child in brief_summary:
                    docs_brief_summary[temp_id]=child.text.strip()
                    # ids_brief_summary.append(temp_id)
                    id_sum=doc_id.text
            if temp_id!=id_sum:
                
                docs_brief_summary[temp_id]=docs_brief_title[temp_id]


            for detailed_description in root.iter('detailed_description'):
                for child in detailed_description:
                    docs_detailed_description[temp_id]=child.text.strip()
                    # ids_detailed_description.append(temp_id)
                    id_detailed=doc_id.text

            if temp_id!=id_detailed:
                
                docs_detailed_description[temp_id]=docs_brief_title[temp_id]+docs_brief_summary[temp_id]


        

            for criteria in root.iter('criteria'):
                for child in criteria:
                    id_crit=doc_id.text
                    doc=child.text.strip().split("Exclusion")
                    doc=doc[0].split()
                    temp=[]
                    for x in doc:
                        if x=="-" or x=="Inclusion" or x=="Criteria:":
                            continue
                        temp.append(x)
                    doc=' '.join(temp)
                    if len(doc)==0:
                        docs_criteria[temp_id]=docs_detailed_description[temp_id]
                    else:
                        docs_criteria[temp_id]=doc

                    # ids_criteria.append(temp_id)
            if temp_id!=id_crit:
                
                docs_criteria[temp_id]=docs_detailed_description[temp_id]

                    
            
    print(f"Number of trials to use: {i}")

    pickle.dump(trials,open("data/trials.bin","wb"))
    pickle.dump(docs_brief_title, open( "data/docs_brief_title.bin", "wb" ) )
    pickle.dump(ids_brief_title, open( "data/doc_ids.bin", "wb" ) )
    pickle.dump(docs_brief_summary,open( "data/docs_brief_summary.bin", "wb" ))
    pickle.dump(docs_detailed_description,open( "data/docs_detailed_description.bin", "wb" ))
    pickle.dump(docs_criteria,open( "data/docs_criteria.bin", "wb" ))
    pickle.dump(ids_brief_summary, open( "data/doc_ids_summary.bin", "wb" ) )
    pickle.dump(ids_detailed_description, open( "data/doc_ids_description.bin", "wb" ) )
    pickle.dump(ids_criteria, open( "data/doc_ids_criteria.bin", "wb" ) )
else:
    trials=pickle.load(open("data/trials.bin","rb"))
    docs_brief_title = pickle.load( open( "data/docs_brief_title.bin", "rb" ) )
    ids_brief_title = pickle.load( open( "data/doc_ids.bin", "rb" ) )
    docs_detailed_description=pickle.load( open( "data/docs_detailed_description.bin", "rb" ) )
    docs_brief_summary=pickle.load( open( "data/docs_brief_summary.bin", "rb" ) )
    docs_criteria=pickle.load( open( "data/docs_criteria.bin", "rb" ) )
    # ids_detailed_description=pickle.load( open( "data/doc_ids_description.bin", "rb" ) )
    # ids_brief_summary=pickle.load( open( "data/doc_ids_summary.bin", "rb" ) )
    # ids_criteria=pickle.load( open( "data/doc_ids_criteria.bin", "rb" ) )

Number of trials to use: 3626


In [3]:

docs_brief_summary=list(docs_brief_summary.values())
docs_brief_title=list(docs_brief_title.values())
docs_detailed_description=list(docs_detailed_description.values())
docs_criteria=list(docs_criteria.values())
print(len(docs_brief_summary),type(docs_brief_summary))
print(len(docs_brief_title),type(docs_brief_title))
print(len(docs_detailed_description),type(docs_detailed_description))
print(len(docs_criteria),type(docs_criteria))
print(len(trials),type(trials))





3626 <class 'list'>
3626 <class 'list'>
3626 <class 'list'>
3626 <class 'list'>
3870 <class 'pandas.core.frame.DataFrame'>


In [5]:
# doc=docs_criteria[1]
# doc=doc.split("Exclusion")
# doc=doc[0].split()
# temp=[]
# for x in doc:
#     if x=="-" or x=="Inclusion" or x=="Criteria:":
#         continue
#     temp.append(x)
# doc=' '.join(temp)
# print(doc)

### Loading the queries 

In [4]:
Queries = "data/topics-2014_2015-summary.topics"
with open(Queries, 'r') as queries_reader:
    txt = queries_reader.read()
root = ET.fromstring(txt)
cases = {}
for q in root.iter('TOP'):
    q_number = q.find("NUM").text
    q_title = q.find('TITLE').text
    cases[q_number] = q_title

### Class for functions used by Language Model with Jelineck-Mercer smooting and Vector Space Model


In [5]:
class NLP:
    def __init__(self, eval_table: pd.DataFrame, cases: dict, ngram_range, docs_brief_title, ids):
        # self.docs_brief_title = documents
        self.eval_table = eval_table
        self.docs_brief_title = docs_brief_title
        self.ids = ids
        self.cases = cases
        self.vsm = TfidfVectorizer(ngram_range=ngram_range, analyzer='word', stop_words = None)
        self.X = self.vsm.fit_transform(docs_brief_title)
        self.vectorizer = CountVectorizer(ngram_range=(1,1), analyzer='word', stop_words = None)
        self.Y=self.vectorizer.fit_transform(docs_brief_title)

    def test_vsm(self, query):
        tfidf = self.vsm.transform([query])
        score = 1 - pairwise_distances(self.X, tfidf, metric='cosine')
        for x in range(len(score)):
            score[x]=float(score[x])
        results = pd.DataFrame(list(zip(self.ids, score)), columns=['_id', 'score'])
        ordered_results = results.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        return ordered_results,results

    def eval(self, result: pd.DataFrame, id):
        def_sim = self.eval_table[self.eval_table["query_id"] == int(id)]
        def_sim = def_sim[def_sim["rel"] != 0]
        rel_ids = def_sim["docid"]
        num_of_rel = def_sim["rel"].count()
        num_of_results = result["_id"].count()

        if num_of_rel == 0:
            return [0, 0, 0, 0, 0]

        #precision@10
        top10 = result['_id'][:10]
        p10 = np.intersect1d(top10, rel_ids).size / 10

        #recall
        recall = np.size(np.intersect1d(result["_id"][:100], rel_ids)) / num_of_rel
        rel_res_vector = np.zeros((num_of_results,))
        for index, row in def_sim.iterrows():
            rel_res_vector = rel_res_vector + ((result['_id'] == row.docid)*row.rel).to_numpy()
        #ndcg5
        ndcg5 = metrics.ndcg_at_k(r = rel_res_vector, k = 5, method = 1)
        ap = metrics.average_precision(rel_res_vector, num_of_rel)
        mrr = metrics.mean_reciprocal_rank(rel_res_vector)

        return [p10, 1 - recall, ndcg5, ap, mrr]

    def evalPR(self, scores, query_id):

        aux = self.eval_table.loc[self.eval_table['query_id'] == int(query_id)]
        idx_rel_docs_brief_title = aux.loc[aux['rel'] != (0)]

        [dummyA, rank_rel, dummyB] = np.intersect1d(scores['_id'], idx_rel_docs_brief_title['docid'], return_indices=True)
        rank_rel = np.sort(rank_rel) + 1
        total_relv_ret = rank_rel.shape[0]
        if total_relv_ret == 0:
            return [np.zeros(11, ), [], total_relv_ret]

        recall = np.arange(1, total_relv_ret + 1) / idx_rel_docs_brief_title.shape[0]
        precision = np.arange(1, total_relv_ret + 1) / rank_rel

        precision_interpolated = np.maximum.accumulate(precision)
        recall_11point = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        precision_11point = np.interp(recall_11point, recall, precision)

        if False:
            print(total_relv_ret)
            print(rank_rel)
            print(recall)
            print(precision)
            plt.plot(recall, precision, color='b', alpha=1)  # Raw precision-recall
            plt.plot(recall, precision_interpolated, color='r', alpha=1)  # Interpolated precision-recall
            plt.plot(recall_11point, precision_11point, color='g', alpha=1)  # 11-point interpolated precision-recall

        return [precision_11point, recall_11point, total_relv_ret]


    def words_and_indexes_association(self,query):
        q_words=[]
        word_idx=[]
        all_words=self.vectorizer.get_feature_names()
        for x in range(len(query)):
            for y in range(len(all_words)):
                if query[x] == all_words[y]:
                    q_words.append(query[x])
                    word_idx.append(y)
                    break
        return word_idx

    def JMS(self):
        x_array=self.Y.toarray()
        x_array=np.array(x_array)
        #sum of rows
        p1=np.sum(x_array,axis=0)
        #sum of cols
        p2=np.sum(x_array,axis=1)
        #number of words in whole corpus
        p3=np.sum(x_array)
        #probability of term in corpus
        matrix_corpus=np.divide(p1,p3)
        #probability of term in document
        p2=np.reshape(p2,(p2.size,1))
        matrix_documents=np.divide(x_array,p2)  
        return matrix_corpus,matrix_documents

    def query_formating(self,query):
        query=query[:-11]
        for y in query:
                if y in ".,":
                    query=query.replace(y,'')
                
        return query 

    def prob_with_JMS(self,index_word,index_doc,lamb,matrix_corp,matrix_doc):
        prob=0
        for x in index_word:
            #print(x,index_doc)
        
            p_md=matrix_doc[index_doc][x]
            p_mc=matrix_corp[x]
            prob+=math.log(lamb*p_md+(1-lamb)*p_mc)
        return prob

    def JSM_test(self,query,lamb,matrix_corpus,matrix_documents):
        scores_now=[]
        q=query.split()
        word_idx=self.words_and_indexes_association(q)
        #print(len(self.ids))
        for x in range(len(self.ids)):
            #print(word_idx,x,lamb)
            prob=self.prob_with_JMS(word_idx,x,lamb,matrix_corpus,matrix_documents)
            scores_now.append(prob)
        results = pd.DataFrame(list(zip(self.ids, scores_now)), columns=['_id', 'score'])
        ordered_results = results.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        return ordered_results,results
    



# Loading the dataset split

In [6]:
cases_training=pickle.load(open("data/cases_training.bin","rb"))
cases_test=pickle.load(open("data/cases_test.bin","rb"))

# Computing all of the scores for labelled pair (query, document)

In [8]:
def matrix_x_creation(cases_training):
    dick={}
    trials=pickle.load(open("data/trials.bin","rb"))
    for x in trials.iterrows():
        # for each row i qrels file dict[query].append all of the documents that are labelled
        dick[str(x[1][0])]=[]
        dick[str(x[1][0])].append(str(x[1][1]))    
    # print(cases_training.items)
    fields=[docs_brief_title,docs_brief_summary,docs_detailed_description,docs_criteria]
    fields_string=["docs_brief_title","docs_brief_summary","docs_detailed_description","docs_criteria"]
    lamb=[0.15,0.25,0.4,0.15]
    final_scores=pd.DataFrame()
    #for each query training set
    for x in cases_training.items():

        scores=pd.DataFrame()
        #if query is qrels file
        if x[0] in dick:
            #get list of documets that are labelled
            temp=dick[x[0]]
            relscores=pd.DataFrame()
            #for each document field
            for y in range(len(fields)):

                myNLP=NLP(trials, cases_training, (1,1), fields[y], ids_brief_title)
                matrix_corpus,matrix_documents=myNLP.JMS()
                query=myNLP.query_formating(x[1])
                #calculate the vsm results
                results_vsm,unordered_results_vsm= myNLP.test_vsm(query)
                #calculate the jms results
                results_jms,unordered_results_jms=myNLP.JSM_test(query,lamb[y],matrix_corpus,matrix_documents)
                temp1=[]
                temp2=[]
                l=[]
                #add column pair in which we have tuple (document id, query id)
                for z in unordered_results_vsm["_id"]:
                    l.append((z,str(x[0])))
                scores["pair"]=l
                #add column with field name+vsm or jms ending with corresponding scores
                scores[fields_string[y]+" vsm"]=unordered_results_vsm["score"]
                scores[fields_string[y]+" jms"]=unordered_results_jms["score"]
            #for each row in these results find these that are labelled in qrels file and put them in relscores dataframe
            for z in scores.iterrows():
                temp1=pd.DataFrame()
                if z[1][0][0] in temp:
                    
                    temp1=z[1]
                    temp1=temp1.to_frame().transpose()
                
                    relscores=pd.concat([relscores,temp1])
            
            #for each query add the labelled results to the final scores dataframe
            final_scores=pd.concat([relscores,final_scores])
        final_scores
        pickle.dump(final_scores,open("data/final_relevant_scores.bin","wb"))
    return final_scores
        


    

        


# Matrix Y definition

In [9]:
#based on matrix y (final results dataframe) we make a matrix y in which we have binary labels for each 
# (query, document) pair from the previous matrix
def matrix_y_def():
    list_of_relevances=[]
    binary_relevance=[]
    final_scores=pickle.load(open("data/final_relevant_scores.bin","rb"))
    trials=pickle.load(open("data/trials.bin","rb"))

    for y in final_scores.iterrows():
            for x in trials.iterrows():
                #here we use the pairs column to find the corresponding label in qrels file
                if (x[1][1],str(x[1][0]))==y[1][0]:
                    z=x[1][2]
                    #if the label is equal to 2 we change it to 1 
                    list_of_relevances.append(z) 
                    if z>0:
                        z=1
                        
                    
                    binary_relevance.append(z)          
    # print(len(list_of_relevances))
    pickle.dump(list_of_relevances,open("data/matrix_y.bin","wb"))
    return list_of_relevances,binary_relevance


# Matrix X normalization 

In [10]:
def matrix_x_norm():
#here we use the min max scaler for normalising obtained scores in matrix x 
    final_scores=pickle.load(open("data/final_relevant_scores.bin","rb"))
    # print(final_scores.shape)
    del final_scores["pair"]
    scaler=MinMaxScaler()
    final_scores=scaler.fit_transform(final_scores)
    # print(list_of_relevances)
    # print(final_scores.shape)
    # print(len(list_of_relevances))
    pickle.dump(final_scores,open("data/matrix_x.bin","wb"))
    # matrix_y=pickle.load(open("data/matrix_y.bin","rb"))

# Logistic Regression 
## using X and Y matrices

In [11]:
def log_regr_train(c,typ,matrix_y,rel):
    #using both matrices with logistic regression
    # matrix_y=pickle.load(open("data/matrix_y.bin","rb"))
    matrix_x=pickle.load(open("data/matrix_x.bin","rb"))
    if typ=="a":
        clf = LogisticRegression(C=c,class_weight=None).fit(matrix_x, matrix_y, sample_weight=None)
    if typ=="b":
        clf = LogisticRegression(C=c,class_weight="balanced").fit(matrix_x, matrix_y, sample_weight=None)
    if typ=="c":
        clf = LogisticRegression(C=c,class_weight=None).fit(matrix_x, matrix_y, sample_weight=rel)
    #coeficients correspond to weight we should give each column in matrix X
    # (how important are the scores obtaned from each document field)
    return clf.coef_

# Computing the scores using logistic regression weights

In [12]:
def calc_scores_with_coef(coef,cases,lamb,c):    
    #here we compute the scores and use metrics to evaluate them
    #but each scores list is multiplied by corresponding logistic regression coeficient 
    # coef=clf.coef_
    # mock_wieghts=[[1.0,0,0,0,0,0,0,0]]
    records = []
    fields=[docs_brief_title,docs_brief_summary,docs_detailed_description,docs_criteria]
    fields_string=["docs_brief_title","docs_brief_summary","docs_detailed_description","docs_criteria"]
    # lamb=[0.15,0.25,0.4,0.15]
    prec = np.zeros(11,)
    test = np.zeros(5,)

    for x in cases.items():
        scores=pd.DataFrame()
        relscores=pd.DataFrame()
        statistics_letor={}
        coef_vsm=0
        coef_jsm=1
        score_mult_coef=pd.DataFrame()
        zeros=np.zeros(len(ids_brief_title))
        score_mult_coef["score"]=zeros 
        relscores["score"]=zeros
        for y in range(len(fields)):
            
            myNLP=NLP(trials, cases, (1,1), fields[y], ids_brief_title)
            query=myNLP.query_formating(x[1]) 
            matrix_corpus,matrix_documents=myNLP.JMS()
            results_vsm,unordered_results_vsm= myNLP.test_vsm(query)
            unordered_results_vsm["score"]=unordered_results_vsm["score"].multiply(coef[0][coef_vsm])
            # unordered_results_vsm["score"]=unordered_results_vsm["score"].multiply(mock_wieghts[0][coef_vsm])
            results_jms,unordered_results_jms=myNLP.JSM_test(query,lamb[y],matrix_corpus,matrix_documents)
            unordered_results_jms["score"]=unordered_results_jms["score"].multiply(coef[0][coef_jsm])
            # unordered_results_jms["score"]=unordered_results_jms["score"].multiply(mock_wieghts[0][coef_jsm])
            #print(results_vsm)
            # score_mult_coef["_id"]=results_vsm["_id"]
            score_mult_coef[fields_string[y]+" jms"]=unordered_results_vsm["score"].multiply(coef[0][coef_vsm])
            score_mult_coef[fields_string[y]+" vsm"]=unordered_results_jms["score"].multiply(coef[0][coef_jsm])
            coef_jsm+=2
            coef_vsm+=2
            
            # score_mult_coef["score"]=score_mult_coef["score"]+ unordered_results_jms["score"]+unordered_results_vsm["score"]
        relscores["score"]=score_mult_coef.sum(axis=1)
        relscores["_id"]=unordered_results_jms["_id"]
        # score_mult_coef=score_mult_coef.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        relscores=relscores.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        p10, recall, ndcg5, ap, mrr = myNLP.eval(relscores, x[0])
        test +=np.array([p10, ndcg5, mrr, ap, recall])
        prec_11point, recal_11point, tot_relv_ret = myNLP.evalPR(relscores, x[0])
        prec +=prec_11point

    prec = prec / (len(cases.keys()))
    test = test / (len(cases.keys()))
    for p, r in zip(prec, recal_11point):
        records.append((c, p, r))
    statistics_letor["C="+str(c)]=test
    
    stats=pd.DataFrame.from_dict(statistics_letor, orient="index", columns=["p10", "ndcg5", "mrr", "ap", "recall"])
    return stats,records
            
        

# Tuning the JMS lambda for different fields 

In [12]:
fields_string=["docs_brief_title","docs_brief_summary","docs_detailed_description","docs_criteria"]
fields=[docs_brief_title,docs_brief_summary,docs_detailed_description,docs_criteria]
records = []
statistics = {}
for y in range(len(fields)):
    myNLP = NLP(trials, cases_training, (1, 1), fields[y], ids_brief_title)
    records_jms = []
    statistics_jms = {}
    matrix_corpus,matrix_documents=myNLP.JMS()
    for lamb in [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.99]:
        prec = np.zeros(11,)
        test = np.zeros(5,)

        for x in myNLP.cases.items():
            query=myNLP.query_formating(x[1])
            results,unordered_results=myNLP.JSM_test(query,lamb,matrix_corpus,matrix_documents)
            p10,recall,ndcg5,ap,mrr=myNLP.eval(results,x[0])
            test = test + np.array([p10, ndcg5, mrr, ap, recall])
        test = test / len(cases_training.keys())
        statistics_jms[lamb] = test
        # print(lamb)
    stats = pd.DataFrame.from_dict(statistics_jms, orient="index", columns=["p10", "ndcg5", "mrr", "ap", "recall"])
    print(fields_string[y])
    print(stats)

docs_brief_title
           p10     ndcg5       mrr        ap    recall
0.00  0.002083  0.007066  0.005068  0.007654  0.955837
0.05  0.075000  0.062876  0.005068  0.050237  0.795584
0.10  0.081250  0.062833  0.005068  0.049541  0.798172
0.15  0.077083  0.068278  0.005068  0.050040  0.801853
0.20  0.075000  0.067068  0.005068  0.048928  0.798110
0.25  0.072917  0.068596  0.005068  0.048517  0.796694
0.30  0.070833  0.067229  0.005068  0.047326  0.799379
0.35  0.070833  0.065283  0.005068  0.046501  0.798188
0.40  0.070833  0.068164  0.005068  0.045743  0.802356
0.45  0.070833  0.065625  0.005068  0.044557  0.805070
0.50  0.068750  0.064259  0.005068  0.043893  0.804915
0.55  0.062500  0.060012  0.005068  0.042217  0.810868
0.60  0.060417  0.061033  0.005068  0.042016  0.811452
0.65  0.060417  0.061184  0.005068  0.041459  0.813014
0.70  0.060417  0.061337  0.005068  0.040802  0.817800
0.75  0.060417  0.057986  0.005068  0.039881  0.818079
0.80  0.060417  0.058582  0.005068  0.039732  0.

#    Tuning the VSM ngrams

In [13]:
#We are tuning using the training set
fields=[docs_brief_title,docs_brief_summary,docs_detailed_description,docs_criteria]
fields_string=["docs_brief_title","docs_brief_summary","docs_detailed_description","docs_criteria"]
records = []
statistics = {}
results=[]
for y in range(len(fields)):
    for ngram in [(1,1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3)]:
        myNLP = NLP(trials, cases_training, ngram, fields[y], ids_brief_title)
        prec = np.zeros(11,)
        test = np.zeros(5,)
        for x in myNLP.cases.items():
            results,unordered_results= myNLP.test_vsm(x[1])
            p10, recall, ndcg5, ap, mrr = myNLP.eval(results, x[0])
            test = test + np.array([p10, ndcg5, mrr, ap, recall])
            prec_11point, recal_11point, tot_relv_ret = myNLP.evalPR(results, x[0])
            prec = prec + prec_11point
            
        prec = prec / len(cases_training.keys())
        test = test / len(cases_training.keys())
        statistics[ngram] = test
    stats = pd.DataFrame.from_dict(statistics, orient="index", columns=["p10", "ndcg5", "mrr", "ap", "recall"])
    print(fields_string[y])
    print(stats)

docs_brief_title
             p10     ndcg5       mrr        ap    recall
(1, 1)  0.079167  0.073697  0.005068  0.048426  0.803329
(1, 2)  0.064583  0.068421  0.005068  0.045635  0.809815
(2, 2)  0.043750  0.057167  0.005068  0.026414  0.920451
(1, 3)  0.064583  0.065032  0.005068  0.042952  0.810904
(2, 3)  0.043750  0.054010  0.005068  0.025906  0.919324
(3, 3)  0.010417  0.020448  0.005068  0.011434  0.951259
docs_brief_summary
             p10     ndcg5       mrr        ap    recall
(1, 1)  0.097917  0.091399  0.005068  0.075581  0.740466
(1, 2)  0.093750  0.097353  0.005068  0.073953  0.737067
(2, 2)  0.064583  0.072868  0.005068  0.042347  0.848290
(1, 3)  0.091667  0.095186  0.005068  0.072219  0.736217
(2, 3)  0.070833  0.066741  0.005068  0.041246  0.848290
(3, 3)  0.041667  0.050273  0.005068  0.023246  0.931056
docs_detailed_description
             p10     ndcg5       mrr        ap    recall
(1, 1)  0.114583  0.107543  0.005068  0.082028  0.674937
(1, 2)  0.106250  0.113619

## Lambdas we have chosen 
For brief title: 0.10
For brief summary: 0.25
For detailed description: 0.25
For criteria: 0.15
For all fields the best range of n-grams analised was (1,1)


# Tuning the regularization parameter

# A)

In [15]:
ids_training_cases=list(cases_training.keys())
final_stats=pd.DataFrame()
for c in [0.1,0.3,0.5,0.7,0.9]:
    kf = KFold(4, shuffle=True, random_state=23)
    stats=np.zeros((1,5))
    for id_train,id_valid in kf.split(ids_training_cases,ids_training_cases):
        training_set={}
        temp=[]
        validation_set={}
        for id in id_train:
            temp=list(cases_training.keys())
            training_set[temp[id]] = cases_training[temp[id]]
        for id in id_valid:
            id=int(id)
            validation_set[temp[id]] = cases_training[temp[id]]
        # print("zaczął tworzyć x")
        matrix_x=matrix_x_creation(training_set)
        # print("zaczął tworzyć y")
        matrix_y,binary=matrix_y_def()
        matrix_x=matrix_x_norm()
        coef=log_regr_train(c,"a",binary,matrix_y)
        # print("Stats")
        temp,records=calc_scores_with_coef(coef,validation_set,[0.10,0.25,0.25,0.15],c)
        stats+=temp
        # print("_____________")
    print("Stats for c=",c)
    stats=stats/4
    final_stats=pd.concat([final_stats,stats])
    print(stats)
pickle.dump(final_stats,open("data/letor_a","wb"))       



Stats for c= 0.1
           p10     ndcg5       mrr        ap    recall
C=0.1  0.13125  0.126032  0.005068  0.106868  0.655354
Stats for c= 0.3
            p10  ndcg5       mrr        ap    recall
C=0.3  0.133333  0.119  0.005068  0.104456  0.657068
Stats for c= 0.5
            p10     ndcg5       mrr        ap    recall
C=0.5  0.133333  0.121567  0.005068  0.104099  0.656417
Stats for c= 0.7
            p10     ndcg5       mrr        ap    recall
C=0.7  0.135417  0.125107  0.005068  0.103865  0.658949
Stats for c= 0.9
            p10     ndcg5       mrr        ap    recall
C=0.9  0.133333  0.125352  0.005068  0.104281  0.655892


In [16]:
final_stats

,p10,ndcg5,mrr,ap,recall
C=0.1,0.131250,0.126032,0.005068,0.106868,0.655354
C=0.3,0.133333,0.119000,0.005068,0.104456,0.657068
C=0.5,0.133333,0.121567,0.005068,0.104099,0.656417
C=0.7,0.135417,0.125107,0.005068,0.103865,0.658949
C=0.9,0.133333,0.125352,0.005068,0.104281,0.655892


# B)

In [18]:
final_stats=pd.DataFrame()
for c in [0.1,0.3,0.5,0.7,0.9]:
    kf = KFold(4, shuffle=True, random_state=23)
    stats=np.zeros((1,5))
    for id_train,id_valid in kf.split(ids_training_cases,ids_training_cases):
        training_set={}
        temp=[]
        validation_set={}
        for id in id_train:
            temp=list(cases_training.keys())
            training_set[temp[id]] = cases_training[temp[id]]
        for id in id_valid:
            id=int(id)
            validation_set[temp[id]] = cases_training[temp[id]]
        # print("zaczął tworzyć x")
        matrix_x=matrix_x_creation(training_set)
        # print("zaczął tworzyć y")
        matrix_y,binary=matrix_y_def()
        matrix_x=matrix_x_norm()
        coef=log_regr_train(c,"b",binary,matrix_y)
        # print("Stats")
        temp,records=calc_scores_with_coef(coef,validation_set,[0.10,0.25,0.25,0.15],c)
        stats+=temp
        # print(c)
        # print(temp)
        # print("_____________")
    print("Stats for c=",c)
    stats=stats/4
    final_stats=pd.concat([final_stats,stats])
    print(stats)
pickle.dump(final_stats,open("data/letor_b","wb")) 
        

Stats for c= 0.1
            p10     ndcg5       mrr        ap    recall
C=0.1  0.129167  0.124569  0.005068  0.106156  0.655354
Stats for c= 0.3
            p10     ndcg5       mrr        ap    recall
C=0.3  0.133333  0.118738  0.005068  0.103443  0.666794
Stats for c= 0.5
           p10     ndcg5       mrr        ap    recall
C=0.5  0.13125  0.125417  0.005068  0.103605  0.662839
Stats for c= 0.7
            p10     ndcg5       mrr        ap    recall
C=0.7  0.129167  0.128395  0.005068  0.104218  0.661179
Stats for c= 0.9
            p10     ndcg5       mrr        ap    recall
C=0.9  0.133333  0.127486  0.005068  0.105057  0.653602


In [19]:
final_stats

,p10,ndcg5,mrr,ap,recall
C=0.1,0.129167,0.124569,0.005068,0.106156,0.655354
C=0.3,0.133333,0.118738,0.005068,0.103443,0.666794
C=0.5,0.131250,0.125417,0.005068,0.103605,0.662839
C=0.7,0.129167,0.128395,0.005068,0.104218,0.661179
C=0.9,0.133333,0.127486,0.005068,0.105057,0.653602


# C)

In [20]:
final_stats=pd.DataFrame()
for c in [0.1,0.3,0.5,0.7,0.9]:
    kf = KFold(4, shuffle=True, random_state=23)
    stats=np.zeros((1,5))
    for id_train,id_valid in kf.split(ids_training_cases,ids_training_cases):
        temp=[]
        training_set={}
        validation_set={}
        for id in id_train:
            temp=list(cases_training.keys())
            training_set[temp[id]] = cases_training[temp[id]]
        for id in id_valid:
            id=int(id)
            validation_set[temp[id]] = cases_training[temp[id]]
        # print("zaczął tworzyć x")
        matrix_x=matrix_x_creation(training_set)
        # print("zaczął tworzyć y")
        matrix_y,binary=matrix_y_def()
        matrix_x=matrix_x_norm()
        coef=log_regr_train(c,"c",binary,matrix_y)
        # print("Stats")
        temp,records=calc_scores_with_coef(coef,validation_set,[0.10,0.25,0.25,0.15],c)
        stats+=temp
        # print(c)
        # print(temp)
        # print("_____________")
    print("Stats for c=",c)
    stats=stats/4
    final_stats=pd.concat([final_stats,stats])
    print(stats)
pickle.dump(final_stats,open("data/letor_c","wb")) 


Stats for c= 0.1
            p10     ndcg5       mrr        ap    recall
C=0.1  0.145833  0.158025  0.005068  0.111363  0.663758
Stats for c= 0.3
            p10     ndcg5       mrr        ap    recall
C=0.3  0.147917  0.140698  0.005068  0.110669  0.677733
Stats for c= 0.5
            p10     ndcg5       mrr        ap    recall
C=0.5  0.133333  0.134285  0.005068  0.095339  0.679572
Stats for c= 0.7
            p10    ndcg5       mrr        ap    recall
C=0.7  0.141667  0.15992  0.005068  0.108359  0.672437
Stats for c= 0.9
          p10     ndcg5       mrr        ap    recall
C=0.9  0.1375  0.134021  0.005068  0.103732  0.664615


In [21]:
final_stats

,p10,ndcg5,mrr,ap,recall
C=0.1,0.145833,0.158025,0.005068,0.111363,0.663758
C=0.3,0.147917,0.140698,0.005068,0.110669,0.677733
C=0.5,0.133333,0.134285,0.005068,0.095339,0.679572
C=0.7,0.141667,0.159920,0.005068,0.108359,0.672437
C=0.9,0.137500,0.134021,0.005068,0.103732,0.664615


# Evaluation of model a

In [142]:

matrix_x=matrix_x_creation(cases_test)
matrix_x=matrix_x_norm()
matrix_x=pickle.load(open("data/matrix_x.bin","rb"))
matrix_y,binary=matrix_y_def()
coef=log_regr_train(0.1,"a",binary,matrix_y)
print(coef)
stats,records=temp=calc_scores_with_coef(coef,cases_test,[0.10,0.25,0.25,0.15],0.7)
pickle.dump(stats,open("data/letor_a_test_stats","wb")) 
pickle.dump(records,open("data/letor_a_test_records","wb")) 

[[-0.00548344  0.01690746  0.03142435  0.03545309  0.01936886  0.03388526
   0.03758225  0.0252886 ]]


In [134]:
records_comparison=pd.DataFrame()
records=pickle.load(open("data/letor_a_test_records","rb"))
records = pd.DataFrame.from_records(records, columns=["Model", "precision_11", "recall_11"])
records["Model"]="LETOR_A_C=0.7"
stats=pickle.load(open("data/letor_a_test_stats","rb"))
stats
records_comparison=pd.concat([records_comparison,records])

In [127]:
stats

,p10,ndcg5,mrr,ap,recall
C=0.7,0.1,0.143132,0.005148,0.077377,0.653355


In [100]:
fig_a = px.line(records, x="recall_11", y="precision_11", color='Model')
fig_a.show()

# Evaluation of model b

In [143]:
matrix_x=matrix_x_creation(cases_test)
matrix_x=matrix_x_norm()
matrix_y,binary=matrix_y_def()
coef=log_regr_train(0.5,"b",binary,matrix_y)
print(coef)
stats,records=calc_scores_with_coef(coef,cases_test,[0.10,0.25,0.25,0.15],0.9)
pickle.dump(stats,open("data/letor_b_test_stats","wb")) 
pickle.dump(records,open("data/letor_b_test_records","wb")) 

[[-0.06293774  0.07839942  0.1591309   0.18415634  0.09287889  0.17437395
   0.22831373  0.12575199]]


In [135]:
records=pickle.load(open("data/letor_b_test_records","rb"))
records = pd.DataFrame.from_records(records, columns=["Model", "precision_11", "recall_11"])
records["Model"]="LETOR_B_C=0.9"
stats=pickle.load(open("data/letor_b_test_stats","rb"))
stats
records_comparison=pd.concat([records_comparison,records])

In [124]:
stats

,p10,ndcg5,mrr,ap,recall
C=0.9,0.1,0.144983,0.005148,0.076859,0.658721


In [102]:
fig_b = px.line(records, x="recall_11", y="precision_11", color='Model')
fig_b.show()

# Evaluation of model c

In [144]:
matrix_x=matrix_x_creation(cases_test)
matrix_x=matrix_x_norm()
matrix_y,binary=matrix_y_def()
coef=log_regr_train(0.1,"c",binary,matrix_y)
print(coef)
stats,records=calc_scores_with_coef(coef,cases_test,[0.10,0.25,0.25,0.15],0.3)
pickle.dump(stats,open("data/letor_c_test_stats","wb")) 
pickle.dump(records,open("data/letor_c_test_records","wb")) 

[[-2.17325187e-06  6.03064530e-06 -1.91247565e-06  5.45598338e-06
  -4.23324184e-06  4.73201108e-06 -4.12486826e-06  5.31556769e-06]]


In [136]:
records=pickle.load(open("data/letor_c_test_records","rb"))
records = pd.DataFrame.from_records(records, columns=["Model", "precision_11", "recall_11"])
records["Model"]="LETOR_C_C=0.3"
stats=pickle.load(open("data/letor_c_test_stats","rb"))
stats
records_comparison=pd.concat([records_comparison,records])

In [129]:
stats

,p10,ndcg5,mrr,ap,recall
C=0.3,0.1,0.116128,0.005148,0.074141,0.618946


In [104]:
fig_c = px.line(records, x="recall_11", y="precision_11", color='Model')
fig_c.show()

In [105]:
fig = px.line(records_comparison, x="recall_11", y="precision_11", color='Model')
fig.show()

# VSM for different fields


In [137]:
ngrams=[(1,1),(1,1),(1,1),(1,1)]
fields=[docs_brief_title,docs_brief_summary,docs_detailed_description,docs_criteria]
fields_string=["docs_brief_title","docs_brief_summary","docs_detailed_description","docs_criteria"]
lamb=[0.10,0.25,0.25,0.15]
records = []
statistics = {}
for y in range(len(fields)):
    prec = np.zeros(11,)
    test = np.zeros(5,)
    myNLP = NLP(trials, cases_test, ngrams[y], fields[y], ids_brief_title)
    
    for x in myNLP.cases.items():
        results,unordered_results = myNLP.test_vsm(x[1])
        p10, recall, ndcg5, ap, mrr = myNLP.eval(results, x[0])
        test = test + np.array([p10, ndcg5, mrr, ap, recall])
        prec_11point, recal_11point, tot_relv_ret = myNLP.evalPR(results, x[0])
        prec = prec + prec_11point
        
        
        
    prec = prec / len(cases_test.keys())
    test = test / len(cases_test.keys())
    for p, r in zip(prec, recal_11point):
        records.append((fields_string[y]+"_vsm", p, r))

    statistics[fields_string[y]] = test

stats = pd.DataFrame.from_dict(statistics, orient="index", columns=["p10", "ndcg5", "mrr", "ap", "recall"])


In [138]:
stats
records_basic=pd.DataFrame()
records = pd.DataFrame.from_records(records, columns=["Model", "precision_11", "recall_11"])
fig = px.line(records, x="recall_11", y="precision_11", color='Model')
fig.show()
records_comparison=pd.concat([records_comparison,records])
records_basic=pd.concat([records_basic,records])

In [131]:
stats

,p10,ndcg5,mrr,ap,recall
docs_brief_title,0.025000,0.019599,0.005148,0.022378,0.757259
docs_brief_summary,0.083333,0.118670,0.005148,0.056917,0.665127
docs_detailed_description,0.116667,0.140486,0.005148,0.063137,0.678178
docs_criteria,0.100000,0.115121,0.005148,0.057237,0.691091


# JMS for different fields

In [139]:
fields_string=["docs_brief_title","docs_brief_summary","docs_detailed_description","docs_criteria"]
fields=[docs_brief_title,docs_brief_summary,docs_detailed_description,docs_criteria]
lamb=[0.10,0.25,0.25,0.15]
records = []
statistics = {}
for y in range(len(fields)):
    myNLP = NLP(trials, cases_test, (1,1), fields[y], ids_brief_title)
    matrix_corpus,matrix_documents=myNLP.JMS()
    prec = np.zeros(11,)
    test = np.zeros(5,)
    for x in myNLP.cases.items():
        query=myNLP.query_formating(x[1])
        results,unordered_results=myNLP.JSM_test(query,lamb[y],matrix_corpus,matrix_documents)
        p10,recall,ndcg5,ap,mrr=myNLP.eval(results,x[0])
        test = test + np.array([p10, ndcg5, mrr, ap, recall])
        prec_11point, recal_11point, tot_relv_ret = myNLP.evalPR(results, x[0])
        prec = prec + prec_11point
    
    prec = prec / len(cases_test.keys())
    test = test / len(cases_test.keys())
    for p, r in zip(prec, recal_11point):
        records.append((fields_string[y]+"_jms", p, r))

    statistics[fields_string[y]] = test
stats = pd.DataFrame.from_dict(statistics, orient="index", columns=["p10", "ndcg5", "mrr", "ap", "recall"])


In [133]:
stats

,p10,ndcg5,mrr,ap,recall
docs_brief_title,0.033333,0.044670,0.005148,0.033093,0.752188
docs_brief_summary,0.050000,0.088075,0.005148,0.046976,0.684019
docs_detailed_description,0.091667,0.141038,0.005148,0.059305,0.732407
docs_criteria,0.116667,0.175942,0.005148,0.055480,0.697024


In [140]:
stats
records = pd.DataFrame.from_records(records, columns=["Model", "precision_11", "recall_11"])

fig = px.line(records, x="recall_11", y="precision_11", color='Model')
fig.show()
records_comparison=pd.concat([records_comparison,records])
records_basic=pd.concat([records_basic,records])

In [141]:
fig = px.line(records_comparison, x="recall_11", y="precision_11", color='Model')
fig.show()

In [122]:
fig = px.line(records_basic, x="recall_11", y="precision_11", color='Model')
fig.show()